<a href="https://colab.research.google.com/github/dondiana/pp/blob/main/pandas2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Problem 1
import pandas as pd
import numpy as np

def sum_daily_odd_even(transactions: pd.DataFrame) -> pd.DataFrame:
    transactions["bool"] = transactions["amount"]%2 == 0
    transactions["odd"] = np.where(~transactions["bool"], transactions["amount"], 0)
    transactions["even"] = np.where(transactions["bool"], transactions["amount"], 0)
    grouped = transactions.groupby(by="transaction_date", as_index=False).agg(odd_sum=("odd", "sum"), even_sum=("even", "sum"))
    return grouped.sort_values(by="transaction_date", ascending=True)

In [ ]:
#Problem 2
import pandas as pd

def count_unique_subjects(teacher: pd.DataFrame) -> pd.DataFrame:
    return teacher.groupby('teacher_id')[
        'subject_id'
        ].nunique().reset_index().rename({
            'subject_id':'cnt'
        },axis=1)

In [ ]:
#Problem 3
import pandas as pd

def find_primary_department(employee: pd.DataFrame) -> pd.DataFrame:

    single = employee.employee_id.unique()[employee.groupby("employee_id").count()["department_id"] == 1]

    return pd.concat([employee[employee["primary_flag"] == "Y"], employee[employee["employee_id"].isin(single)]])[["employee_id", "department_id"]].drop_duplicates(subset =["employee_id"])

In [ ]:
#Problem 4
import pandas as pd

def rearrange_products_table(products: pd.DataFrame) -> pd.DataFrame:
    df=products.melt(
        id_vars="product_id",
        value_vars=["store1","store2","store3"],
        var_name="store",
        value_name="price"
    )
    df.dropna(subset="price",inplace=True)

    return df


In [ ]:
#Problem 5
import pandas as pd

def calculate_special_bonus(employees: pd.DataFrame) -> pd.DataFrame:

    #defining a dataframe
    df = pd.DataFrame(employees)

    #initializing bonus column as equal to salary column
    df['bonus'] = df['salary']

    #sorting values in the bonus table
    bonus = df[(df['employee_id'] % 2 != 0) & (~df['name'].str.startswith('M'))]

    #creating and sorting values in the non-bonus table
    non_bonus = df[(df['employee_id'] % 2 == 0) | (df['name'].str.startswith('M'))]

    #changing the bonus values to 0
    non_bonus.loc[:, 'bonus'] = 0

    #there's a potential issue with the SettingWithCopyWarning that may arise when you modify non_bonus['bonus'] = 0. This warning appears because pandas creates a "view" of the original DataFrame, and modifying this view can lead to unexpected results. To avoid this warning, you can use .loc to directly assign values.
    df_concat = pd.concat([bonus, non_bonus], axis=0).sort_values(by='employee_id', ascending=True)[['employee_id', 'bonus']]

    #returning table
    return df_concat


In [ ]:
#Problem 6
import pandas as pd

def latest_login(logins: pd.DataFrame) -> pd.DataFrame:
    # create year column
    logins['year']= logins['time_stamp'].dt.year
    filtered = logins[logins['year']==2020]
    grouped = filtered.groupby('user_id')[['time_stamp']].max().reset_index().rename(columns={'time_stamp':'last_stamp'})
    return grouped


In [ ]:
#Problem 7
import pandas as pd

def count_salary_categories(accounts: pd.DataFrame) -> pd.DataFrame:
    data = []
    ls = accounts.loc[accounts["income"] < 20000]
    data.append(["Low Salary", len(ls.axes[0])])
    avs = accounts.loc[(accounts["income"] >= 20000) & (accounts["income"] <= 50000)]
    data.append(["Average Salary", len(avs.axes[0])])
    hs = accounts.loc[accounts["income"] > 50000]
    data.append(["High Salary", len(hs.axes[0])])

    return pd.DataFrame(data, columns = ["category", "accounts_count"])


In [ ]:
#Problem 8
import pandas as pd

def confirmation_rate(signups: pd.DataFrame, confirmations: pd.DataFrame) -> pd.DataFrame:
    confirmations.action = confirmations.action.apply(lambda x: 1 if (x == "confirmed") else 0)

    # Calculate the confirmation rate
    grouped = confirmations.groupby("user_id")["action"].mean().reset_index(name = "confirmation_rate").merge(signups["user_id"], on = "user_id", how = "right").fillna(0)

    grouped["confirmation_rate"] = round(grouped["confirmation_rate"] + 1e-6, 2)

    return grouped